# Environment

In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Local environment
Root dir: /Users/cuiyuting/Desktop/release_popularity_prediction
Added the following directory to the PYTHONPATH: /Users/cuiyuting/Desktop/release_popularity_prediction


In [2]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
project = hopsworks.login()

2026-01-10 23:15:02,425 INFO: Initializing external client
2026-01-10 23:15:02,427 INFO: Base URL: https://c.app.hopsworks.ai:443


2026-01-10 23:15:04,734 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1342613


# Read CSV into Dataframe

In [27]:
today = datetime.date.today()

csv_file=f"{root_dir}/data/release_raw_data.csv"

df_original = pd.read_csv(csv_file, skipinitialspace=True)
df_original

,full_name,repo_stars,repo_forks,repo_watchers,language,repo_created_at,repo_updated_at,topics,release_name,release_body,author_followers,author_public_repos,author_type,published_at,prerelease,draft,first_week_star
0,HANCORE-linux/waybar-themes/v2.1.4,113,1,113,CSS,2025-09-06T22:08:06Z,2026-01-09T15:00:36Z,[],v2.1.4,"# 🎉 New Waybar Release ""V2.1a & V2.1b – Oxoca...",52,21,User,2026-01-08T20:07:56Z,False,False,3
1,HANCORE-linux/waybar-themes/v2.1.3,113,1,113,CSS,2025-09-06T22:08:06Z,2026-01-09T15:00:36Z,[],v2.1.3,## 🔧 Changes\r\n### V3-min3\r\n- Adjusted back...,52,21,User,2026-01-05T23:56:48Z,False,False,11
2,HANCORE-linux/waybar-themes/v2.1.2,113,1,113,CSS,2025-09-06T22:08:06Z,2026-01-09T15:00:36Z,[],v2.1.2,# Changes\r\n#### V3-min3\r\n- reworked entire...,52,21,User,2026-01-03T22:20:14Z,False,False,12
3,HANCORE-linux/waybar-themes/v2.1.1,113,1,113,CSS,2025-09-06T22:08:06Z,2026-01-09T15:00:36Z,[],v2.1.1,"# 🎉 New Waybar Release ""V6.fa""\r\n- Changed f...",52,21,User,2026-01-02T20:03:09Z,False,False,14
4,HANCORE-linux/waybar-themes/v2.1.0,113,1,113,CSS,2025-09-06T22:08:06Z,2026-01-09T15:00:36Z,[],v2.1.0,"# 🎉 New Waybar Release ""V6.f""\r\n- Cycles thr...",52,21,User,2026-01-02T14:25:38Z,False,False,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,whyisdifficult/jiratui/v1.2.0,1367,40,1367,Python,2025-08-04T15:10:45Z,2026-01-09T22:45:43Z,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.2.0,## What's Changed\r\n\r\n* Add completions Sub...,30,1,User,2025-10-04T11:01:46Z,False,False,61
4045,whyisdifficult/jiratui/v1.1.0,1367,40,1367,Python,2025-08-04T15:10:45Z,2026-01-09T22:45:43Z,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.1.0,## What's Changed\r\n\r\n## Added\r\n\r\n* Add...,30,1,User,2025-09-20T13:40:38Z,False,False,32
4046,whyisdifficult/jiratui/v1.0.0,1367,40,1367,Python,2025-08-04T15:10:45Z,2026-01-09T22:45:43Z,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.0.0,## What's Changed\r\n\r\n### Breaking Changes\...,30,1,User,2025-09-16T17:21:47Z,False,False,63
4047,whyisdifficult/jiratui/v0.2.0,1367,40,1367,Python,2025-08-04T15:10:45Z,2026-01-09T22:45:43Z,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v0.2.0,## What's Changed\r\n\r\n### Application\r\n* ...,30,1,User,2025-09-13T10:07:39Z,False,False,125


In [28]:
df_original.dtypes

full_name              object
repo_stars              int64
repo_forks              int64
repo_watchers           int64
language               object
repo_created_at        object
repo_updated_at        object
topics                 object
release_name           object
release_body           object
author_followers        int64
author_public_repos     int64
author_type            object
published_at           object
prerelease               bool
draft                    bool
first_week_star         int64
dtype: object

In [29]:
date_cols = ['repo_created_at', 'repo_updated_at', 'published_at']
for col in date_cols:
    df_original[col] = pd.to_datetime(df_original[col])
df_original.dtypes

full_name                           object
repo_stars                           int64
repo_forks                           int64
repo_watchers                        int64
language                            object
repo_created_at        datetime64[ns, UTC]
repo_updated_at        datetime64[ns, UTC]
topics                              object
release_name                        object
release_body                        object
author_followers                     int64
author_public_repos                  int64
author_type                         object
published_at           datetime64[ns, UTC]
prerelease                            bool
draft                                 bool
first_week_star                      int64
dtype: object

# Feature Engineering

In [74]:
# almost all False in the dataset
df = df_original.drop(columns=['prerelease', 'draft'])
df

,full_name,repo_stars,repo_forks,repo_watchers,language,repo_created_at,repo_updated_at,topics,release_name,release_body,author_followers,author_public_repos,author_type,published_at,first_week_star
0,HANCORE-linux/waybar-themes/v2.1.4,113,1,113,CSS,2025-09-06 22:08:06+00:00,2026-01-09 15:00:36+00:00,[],v2.1.4,"# 🎉 New Waybar Release ""V2.1a & V2.1b – Oxoca...",52,21,User,2026-01-08 20:07:56+00:00,3
1,HANCORE-linux/waybar-themes/v2.1.3,113,1,113,CSS,2025-09-06 22:08:06+00:00,2026-01-09 15:00:36+00:00,[],v2.1.3,## 🔧 Changes\r\n### V3-min3\r\n- Adjusted back...,52,21,User,2026-01-05 23:56:48+00:00,11
2,HANCORE-linux/waybar-themes/v2.1.2,113,1,113,CSS,2025-09-06 22:08:06+00:00,2026-01-09 15:00:36+00:00,[],v2.1.2,# Changes\r\n#### V3-min3\r\n- reworked entire...,52,21,User,2026-01-03 22:20:14+00:00,12
3,HANCORE-linux/waybar-themes/v2.1.1,113,1,113,CSS,2025-09-06 22:08:06+00:00,2026-01-09 15:00:36+00:00,[],v2.1.1,"# 🎉 New Waybar Release ""V6.fa""\r\n- Changed f...",52,21,User,2026-01-02 20:03:09+00:00,14
4,HANCORE-linux/waybar-themes/v2.1.0,113,1,113,CSS,2025-09-06 22:08:06+00:00,2026-01-09 15:00:36+00:00,[],v2.1.0,"# 🎉 New Waybar Release ""V6.f""\r\n- Cycles thr...",52,21,User,2026-01-02 14:25:38+00:00,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,whyisdifficult/jiratui/v1.2.0,1367,40,1367,Python,2025-08-04 15:10:45+00:00,2026-01-09 22:45:43+00:00,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.2.0,## What's Changed\r\n\r\n* Add completions Sub...,30,1,User,2025-10-04 11:01:46+00:00,61
4045,whyisdifficult/jiratui/v1.1.0,1367,40,1367,Python,2025-08-04 15:10:45+00:00,2026-01-09 22:45:43+00:00,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.1.0,## What's Changed\r\n\r\n## Added\r\n\r\n* Add...,30,1,User,2025-09-20 13:40:38+00:00,32
4046,whyisdifficult/jiratui/v1.0.0,1367,40,1367,Python,2025-08-04 15:10:45+00:00,2026-01-09 22:45:43+00:00,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.0.0,## What's Changed\r\n\r\n### Breaking Changes\...,30,1,User,2025-09-16 17:21:47+00:00,63
4047,whyisdifficult/jiratui/v0.2.0,1367,40,1367,Python,2025-08-04 15:10:45+00:00,2026-01-09 22:45:43+00:00,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v0.2.0,## What's Changed\r\n\r\n### Application\r\n* ...,30,1,User,2025-09-13 10:07:39+00:00,125


In [75]:
# Calculate the duration of a repository
df['repo_duration'] = (df['repo_updated_at'] - df['repo_created_at']).dt.days
df = df.drop(columns=['repo_updated_at', 'repo_created_at'])
df

,full_name,repo_stars,repo_forks,repo_watchers,language,topics,release_name,release_body,author_followers,author_public_repos,author_type,published_at,first_week_star,repo_duration
0,HANCORE-linux/waybar-themes/v2.1.4,113,1,113,CSS,[],v2.1.4,"# 🎉 New Waybar Release ""V2.1a & V2.1b – Oxoca...",52,21,User,2026-01-08 20:07:56+00:00,3,124
1,HANCORE-linux/waybar-themes/v2.1.3,113,1,113,CSS,[],v2.1.3,## 🔧 Changes\r\n### V3-min3\r\n- Adjusted back...,52,21,User,2026-01-05 23:56:48+00:00,11,124
2,HANCORE-linux/waybar-themes/v2.1.2,113,1,113,CSS,[],v2.1.2,# Changes\r\n#### V3-min3\r\n- reworked entire...,52,21,User,2026-01-03 22:20:14+00:00,12,124
3,HANCORE-linux/waybar-themes/v2.1.1,113,1,113,CSS,[],v2.1.1,"# 🎉 New Waybar Release ""V6.fa""\r\n- Changed f...",52,21,User,2026-01-02 20:03:09+00:00,14,124
4,HANCORE-linux/waybar-themes/v2.1.0,113,1,113,CSS,[],v2.1.0,"# 🎉 New Waybar Release ""V6.f""\r\n- Cycles thr...",52,21,User,2026-01-02 14:25:38+00:00,15,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,whyisdifficult/jiratui/v1.2.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.2.0,## What's Changed\r\n\r\n* Add completions Sub...,30,1,User,2025-10-04 11:01:46+00:00,61,158
4045,whyisdifficult/jiratui/v1.1.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.1.0,## What's Changed\r\n\r\n## Added\r\n\r\n* Add...,30,1,User,2025-09-20 13:40:38+00:00,32,158
4046,whyisdifficult/jiratui/v1.0.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.0.0,## What's Changed\r\n\r\n### Breaking Changes\...,30,1,User,2025-09-16 17:21:47+00:00,63,158
4047,whyisdifficult/jiratui/v0.2.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v0.2.0,## What's Changed\r\n\r\n### Application\r\n* ...,30,1,User,2025-09-13 10:07:39+00:00,125,158


In [76]:
# one-hot author type
df['org_author'] = (df['author_type'] == 'Organization').astype(int)
df['user_author'] = (df['author_type'] == 'User').astype(int)
df = df.drop(columns=['author_type'])
df

,full_name,repo_stars,repo_forks,repo_watchers,language,topics,release_name,release_body,author_followers,author_public_repos,published_at,first_week_star,repo_duration,org_author,user_author
0,HANCORE-linux/waybar-themes/v2.1.4,113,1,113,CSS,[],v2.1.4,"# 🎉 New Waybar Release ""V2.1a & V2.1b – Oxoca...",52,21,2026-01-08 20:07:56+00:00,3,124,0,1
1,HANCORE-linux/waybar-themes/v2.1.3,113,1,113,CSS,[],v2.1.3,## 🔧 Changes\r\n### V3-min3\r\n- Adjusted back...,52,21,2026-01-05 23:56:48+00:00,11,124,0,1
2,HANCORE-linux/waybar-themes/v2.1.2,113,1,113,CSS,[],v2.1.2,# Changes\r\n#### V3-min3\r\n- reworked entire...,52,21,2026-01-03 22:20:14+00:00,12,124,0,1
3,HANCORE-linux/waybar-themes/v2.1.1,113,1,113,CSS,[],v2.1.1,"# 🎉 New Waybar Release ""V6.fa""\r\n- Changed f...",52,21,2026-01-02 20:03:09+00:00,14,124,0,1
4,HANCORE-linux/waybar-themes/v2.1.0,113,1,113,CSS,[],v2.1.0,"# 🎉 New Waybar Release ""V6.f""\r\n- Cycles thr...",52,21,2026-01-02 14:25:38+00:00,15,124,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,whyisdifficult/jiratui/v1.2.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.2.0,## What's Changed\r\n\r\n* Add completions Sub...,30,1,2025-10-04 11:01:46+00:00,61,158,0,1
4045,whyisdifficult/jiratui/v1.1.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.1.0,## What's Changed\r\n\r\n## Added\r\n\r\n* Add...,30,1,2025-09-20 13:40:38+00:00,32,158,0,1
4046,whyisdifficult/jiratui/v1.0.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.0.0,## What's Changed\r\n\r\n### Breaking Changes\...,30,1,2025-09-16 17:21:47+00:00,63,158,0,1
4047,whyisdifficult/jiratui/v0.2.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v0.2.0,## What's Changed\r\n\r\n### Application\r\n* ...,30,1,2025-09-13 10:07:39+00:00,125,158,0,1


In [77]:
# whether release publish date is weekday or weekend
df['publish_is_weekday'] = (df['published_at'].dt.dayofweek <= 4).astype(int)
df

,full_name,repo_stars,repo_forks,repo_watchers,language,topics,release_name,release_body,author_followers,author_public_repos,published_at,first_week_star,repo_duration,org_author,user_author,publish_is_weekday
0,HANCORE-linux/waybar-themes/v2.1.4,113,1,113,CSS,[],v2.1.4,"# 🎉 New Waybar Release ""V2.1a & V2.1b – Oxoca...",52,21,2026-01-08 20:07:56+00:00,3,124,0,1,1
1,HANCORE-linux/waybar-themes/v2.1.3,113,1,113,CSS,[],v2.1.3,## 🔧 Changes\r\n### V3-min3\r\n- Adjusted back...,52,21,2026-01-05 23:56:48+00:00,11,124,0,1,1
2,HANCORE-linux/waybar-themes/v2.1.2,113,1,113,CSS,[],v2.1.2,# Changes\r\n#### V3-min3\r\n- reworked entire...,52,21,2026-01-03 22:20:14+00:00,12,124,0,1,0
3,HANCORE-linux/waybar-themes/v2.1.1,113,1,113,CSS,[],v2.1.1,"# 🎉 New Waybar Release ""V6.fa""\r\n- Changed f...",52,21,2026-01-02 20:03:09+00:00,14,124,0,1,1
4,HANCORE-linux/waybar-themes/v2.1.0,113,1,113,CSS,[],v2.1.0,"# 🎉 New Waybar Release ""V6.f""\r\n- Cycles thr...",52,21,2026-01-02 14:25:38+00:00,15,124,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,whyisdifficult/jiratui/v1.2.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.2.0,## What's Changed\r\n\r\n* Add completions Sub...,30,1,2025-10-04 11:01:46+00:00,61,158,0,1,0
4045,whyisdifficult/jiratui/v1.1.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.1.0,## What's Changed\r\n\r\n## Added\r\n\r\n* Add...,30,1,2025-09-20 13:40:38+00:00,32,158,0,1,0
4046,whyisdifficult/jiratui/v1.0.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.0.0,## What's Changed\r\n\r\n### Breaking Changes\...,30,1,2025-09-16 17:21:47+00:00,63,158,0,1,1
4047,whyisdifficult/jiratui/v0.2.0,1367,40,1367,Python,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v0.2.0,## What's Changed\r\n\r\n### Application\r\n* ...,30,1,2025-09-13 10:07:39+00:00,125,158,0,1,0


In [78]:
# one-hot programming language
top_n = 20
top_languages = df['language'].value_counts().head(top_n).index
df['language_group'] = df['language'].where(df['language'].isin(top_languages), 'Other')
print("Language Distribution:")
print(df['language_group'].value_counts(normalize=True).head(15))

df = pd.concat([df.drop(['language', 'language_group'], axis=1), 
                pd.get_dummies(df['language_group'], prefix='language_')], axis=1)
df.columns = df.columns.str.replace('C#', 'Csharp').str.replace('C++', 'Cplusplus')
df

Language Distribution:
language_group
TypeScript    0.298345
Python        0.214868
Go            0.076809
JavaScript    0.067177
Rust          0.065942
Other         0.048160
C#            0.036058
Kotlin        0.035564
Swift         0.021487
C             0.019017
Vue           0.016794
Java          0.016547
PHP           0.012349
Shell         0.010620
C++           0.010373
Name: proportion, dtype: float64


,full_name,repo_stars,repo_forks,repo_watchers,topics,release_name,release_body,author_followers,author_public_repos,published_at,...,language__Other,language__PHP,language__Python,language__QML,language__Rust,language__Shell,language__Svelte,language__Swift,language__TypeScript,language__Vue
0,HANCORE-linux/waybar-themes/v2.1.4,113,1,113,[],v2.1.4,"# 🎉 New Waybar Release ""V2.1a & V2.1b – Oxoca...",52,21,2026-01-08 20:07:56+00:00,...,False,False,False,False,False,False,False,False,False,False
1,HANCORE-linux/waybar-themes/v2.1.3,113,1,113,[],v2.1.3,## 🔧 Changes\r\n### V3-min3\r\n- Adjusted back...,52,21,2026-01-05 23:56:48+00:00,...,False,False,False,False,False,False,False,False,False,False
2,HANCORE-linux/waybar-themes/v2.1.2,113,1,113,[],v2.1.2,# Changes\r\n#### V3-min3\r\n- reworked entire...,52,21,2026-01-03 22:20:14+00:00,...,False,False,False,False,False,False,False,False,False,False
3,HANCORE-linux/waybar-themes/v2.1.1,113,1,113,[],v2.1.1,"# 🎉 New Waybar Release ""V6.fa""\r\n- Changed f...",52,21,2026-01-02 20:03:09+00:00,...,False,False,False,False,False,False,False,False,False,False
4,HANCORE-linux/waybar-themes/v2.1.0,113,1,113,[],v2.1.0,"# 🎉 New Waybar Release ""V6.f""\r\n- Cycles thr...",52,21,2026-01-02 14:25:38+00:00,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,whyisdifficult/jiratui/v1.2.0,1367,40,1367,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.2.0,## What's Changed\r\n\r\n* Add completions Sub...,30,1,2025-10-04 11:01:46+00:00,...,False,False,True,False,False,False,False,False,False,False
4045,whyisdifficult/jiratui/v1.1.0,1367,40,1367,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.1.0,## What's Changed\r\n\r\n## Added\r\n\r\n* Add...,30,1,2025-09-20 13:40:38+00:00,...,False,False,True,False,False,False,False,False,False,False
4046,whyisdifficult/jiratui/v1.0.0,1367,40,1367,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.0.0,## What's Changed\r\n\r\n### Breaking Changes\...,30,1,2025-09-16 17:21:47+00:00,...,False,False,True,False,False,False,False,False,False,False
4047,whyisdifficult/jiratui/v0.2.0,1367,40,1367,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v0.2.0,## What's Changed\r\n\r\n### Application\r\n* ...,30,1,2025-09-13 10:07:39+00:00,...,False,False,True,False,False,False,False,False,False,False


In [ ]:
# # Normalization
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# need_scaling = [
#     'repo_stars',
#     'repo_forks',
#     'repo_watchers',
#     'author_followers',
#     'author_public_repos',
#     'repo_duration'
# ]
# scaled_features = scaler.fit_transform(df[need_scaling])
# df_scaled = pd.DataFrame(scaled_features, columns=need_scaling, index=df.index)
# for col in need_scaling:
#     df[col] = df_scaled[col]
# df

,full_name,repo_stars,repo_forks,repo_watchers,topics,release_name,release_body,author_followers,author_public_repos,published_at,...,language__Other,language__PHP,language__Python,language__QML,language__Rust,language__Shell,language__Svelte,language__Swift,language__TypeScript,language__Vue
0,HANCORE-linux/waybar-themes/v2.1.4,-1.824037,-0.368420,-1.824037,[],v2.1.4,"# 🎉 New Waybar Release ""V2.1a & V2.1b – Oxoca...",-0.235755,-0.149384,2026-01-08 20:07:56+00:00,...,False,False,False,False,False,False,False,False,False,False
1,HANCORE-linux/waybar-themes/v2.1.3,-1.824037,-0.368420,-1.824037,[],v2.1.3,## 🔧 Changes\r\n### V3-min3\r\n- Adjusted back...,-0.235755,-0.149384,2026-01-05 23:56:48+00:00,...,False,False,False,False,False,False,False,False,False,False
2,HANCORE-linux/waybar-themes/v2.1.2,-1.824037,-0.368420,-1.824037,[],v2.1.2,# Changes\r\n#### V3-min3\r\n- reworked entire...,-0.235755,-0.149384,2026-01-03 22:20:14+00:00,...,False,False,False,False,False,False,False,False,False,False
3,HANCORE-linux/waybar-themes/v2.1.1,-1.824037,-0.368420,-1.824037,[],v2.1.1,"# 🎉 New Waybar Release ""V6.fa""\r\n- Changed f...",-0.235755,-0.149384,2026-01-02 20:03:09+00:00,...,False,False,False,False,False,False,False,False,False,False
4,HANCORE-linux/waybar-themes/v2.1.0,-1.824037,-0.368420,-1.824037,[],v2.1.0,"# 🎉 New Waybar Release ""V6.f""\r\n- Cycles thr...",-0.235755,-0.149384,2026-01-02 14:25:38+00:00,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,whyisdifficult/jiratui/v1.2.0,-0.485108,-0.307315,-0.485108,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.2.0,## What's Changed\r\n\r\n* Add completions Sub...,-0.237363,-0.178537,2025-10-04 11:01:46+00:00,...,False,False,True,False,False,False,False,False,False,False
4045,whyisdifficult/jiratui/v1.1.0,-0.485108,-0.307315,-0.485108,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.1.0,## What's Changed\r\n\r\n## Added\r\n\r\n* Add...,-0.237363,-0.178537,2025-09-20 13:40:38+00:00,...,False,False,True,False,False,False,False,False,False,False
4046,whyisdifficult/jiratui/v1.0.0,-0.485108,-0.307315,-0.485108,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.0.0,## What's Changed\r\n\r\n### Breaking Changes\...,-0.237363,-0.178537,2025-09-16 17:21:47+00:00,...,False,False,True,False,False,False,False,False,False,False
4047,whyisdifficult/jiratui/v0.2.0,-0.485108,-0.307315,-0.485108,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v0.2.0,## What's Changed\r\n\r\n### Application\r\n* ...,-0.237363,-0.178537,2025-09-13 10:07:39+00:00,...,False,False,True,False,False,False,False,False,False,False


In [79]:
# Check missing values
missing_values = df.isnull().sum()
missing_info = pd.DataFrame({
    'Missing nums': missing_values
})
print(missing_info[missing_info['Missing nums'] > 0])

              Missing nums
release_name            73
release_body           172


In [80]:
# Fill missing values
df_filled = df.copy()
df_filled['release_name'] = df_filled['release_name'].fillna('')
df_filled['release_body'] = df_filled['release_body'].fillna('')
df_filled

,full_name,repo_stars,repo_forks,repo_watchers,topics,release_name,release_body,author_followers,author_public_repos,published_at,...,language__Other,language__PHP,language__Python,language__QML,language__Rust,language__Shell,language__Svelte,language__Swift,language__TypeScript,language__Vue
0,HANCORE-linux/waybar-themes/v2.1.4,113,1,113,[],v2.1.4,"# 🎉 New Waybar Release ""V2.1a & V2.1b – Oxoca...",52,21,2026-01-08 20:07:56+00:00,...,False,False,False,False,False,False,False,False,False,False
1,HANCORE-linux/waybar-themes/v2.1.3,113,1,113,[],v2.1.3,## 🔧 Changes\r\n### V3-min3\r\n- Adjusted back...,52,21,2026-01-05 23:56:48+00:00,...,False,False,False,False,False,False,False,False,False,False
2,HANCORE-linux/waybar-themes/v2.1.2,113,1,113,[],v2.1.2,# Changes\r\n#### V3-min3\r\n- reworked entire...,52,21,2026-01-03 22:20:14+00:00,...,False,False,False,False,False,False,False,False,False,False
3,HANCORE-linux/waybar-themes/v2.1.1,113,1,113,[],v2.1.1,"# 🎉 New Waybar Release ""V6.fa""\r\n- Changed f...",52,21,2026-01-02 20:03:09+00:00,...,False,False,False,False,False,False,False,False,False,False
4,HANCORE-linux/waybar-themes/v2.1.0,113,1,113,[],v2.1.0,"# 🎉 New Waybar Release ""V6.f""\r\n- Cycles thr...",52,21,2026-01-02 14:25:38+00:00,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,whyisdifficult/jiratui/v1.2.0,1367,40,1367,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.2.0,## What's Changed\r\n\r\n* Add completions Sub...,30,1,2025-10-04 11:01:46+00:00,...,False,False,True,False,False,False,False,False,False,False
4045,whyisdifficult/jiratui/v1.1.0,1367,40,1367,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.1.0,## What's Changed\r\n\r\n## Added\r\n\r\n* Add...,30,1,2025-09-20 13:40:38+00:00,...,False,False,True,False,False,False,False,False,False,False
4046,whyisdifficult/jiratui/v1.0.0,1367,40,1367,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v1.0.0,## What's Changed\r\n\r\n### Breaking Changes\...,30,1,2025-09-16 17:21:47+00:00,...,False,False,True,False,False,False,False,False,False,False
4047,whyisdifficult/jiratui/v0.2.0,1367,40,1367,"['atlassian', 'atlassian-jira', 'cli', 'develo...",v0.2.0,## What's Changed\r\n\r\n### Application\r\n* ...,30,1,2025-09-13 10:07:39+00:00,...,False,False,True,False,False,False,False,False,False,False


# Create Feature Groups and insert DataFrames

In [81]:
fs = project.get_feature_store()

In [82]:
release_fg = fs.get_or_create_feature_group(
    name='release_info',
    description='Github release information',
    version=1,
    primary_key=['full_name'],
    event_time="published_at"
)

In [83]:
release_fg.insert(df_filled)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1342613/fs/1331268/fg/1911300


Uploading Dataframe: 100.00% |██████████| Rows 4049/4049 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: release_info_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1342613/jobs/named/release_info_1_offline_fg_materialization/executions


(Job('release_info_1_offline_fg_materialization', 'SPARK'), None)